# cell2func

> Convert desired notebook cells to functions. 

Detects function inputs automatically and function outputs semi-automatically. In the latter case, hints are provided to the developer to refine the list of outputs per each cell.

In [ ]:
#| default_exp core.cell2func

In [ ]:
#| export
import os
import shlex
from dataclasses import dataclass
from functools import reduce
from pathlib import Path
import sys
import ast
from IPython import get_ipython
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)
from IPython.core.magic_arguments import (argument, magic_arguments, parse_argstring)
import ipynbname
from sklearn.utils import Bunch
from fastcore.all import argnames
import nbdev

In [ ]:
#|export
class FunctionProcessor (Bunch):
    def to_file (self, file_path, mode='w'):
        with open (file_path, mode=mode) as file:
            file.write (self.code)
    
    def write (self, file):
        file.write (self.code)
        
    def print (self):
        print (self.code)
    
    def update_code (
        self, 
        arguments=None, 
        return_values=None,
        tab_size=4,
        display=True
    ) -> None:
        if arguments is not None:
            self.arguments = arguments
        arguments = ', '.join (self.arguments)
        if return_values is not None:
            self.return_values = return_values
        return_values = ','.join (self.return_values)
        function_code = ''
        for line in self.original_code.splitlines():
            function_code += f'{" " * tab_size}{line}\n'
        if return_values != '':
            return_line = f'return {return_values}'
            return_line = f'{" " * tab_size}{return_line}\n'
        else:
            return_line = ''
        function_code = f'def {self.name}({arguments}):\n' + function_code + return_line
        self.code = function_code
        get_ipython().run_cell(function_code)
        if display:
            print (function_code)

In [ ]:
#| export
class CellProcessor():
    """
    Processes the cell's code according to the magic command.
    """
    def __init__(self, **kwargs):
        self.function_info = Bunch()
        self.function_list = []
        self.file_name = ipynbname.name().replace ('.ipynb', '.py')
        self.nbs_folder = self.get_nbs_path ()
        self.lib_folder = self.get_lib_path ()
        nb_path = ipynbname.path ()
        index = nb_path.parts.index(self.nbs_folder.name)
        self.file_path = (self.nbs_folder.parent / self.lib_folder.name).joinpath (*nb_path.parts[index+1:])
        
    def cell2file (self, folder, cell):
        folder = Path(folder)
        folder.mkdir(parents=True, exist_ok=True)

        with open(folder / "module.py", "w") as file_handle:
            file_handle.write(cell)

        get_ipython().run_cell(cell)
                    
    def function (
        self,
        func, 
        cell,
        collect_variables_values=True,
        make_function=True,
        tab_size=4,
        update_previous_functions=True,
        discover_outputs=True
    ) -> None:
        
        this_function = FunctionProcessor (
            idx=len(self.function_list), 
            original_code=cell, 
            name=func, 
            discover_outputs=discover_outputs,
            values_before=[]
        )
        if func not in self.function_info:
            self.function_info[func] = this_function
            self.function_list.append (this_function)
            
        idx = this_function.idx
        
        # get variables specific about this function
        if collect_variables_values:
            get_variables_before_code = f'\nkeep_variables ("{func}", "values_before", locals ())'
            get_ipython().run_cell(get_variables_before_code)
            
            get_variables_here_code = cell + f'\nkeep_variables ("{func}", "values_here", locals ())'
            get_ipython().run_cell(get_variables_here_code)
            values_before, values_here = this_function['values_before'], this_function['values_here']
            values_here = {k:values_here[k] for k in set(values_here).difference(values_before)}
            this_function['values_here'] = values_here
            print (values_here)
        
        root = ast.parse (cell)
        variables_here = {node.id for node in ast.walk(root) if isinstance(node, ast.Name) and not callable(eval(node.id))}
        print (variables_here)
        if idx > 0:
            variables_before = reduce (lambda x, y: x['variables_here'] | y['variables_here'], self.function_list[:idx])
        else:
            variables_before = []
        variables_here = sorted (variables_here.difference(variables_before))
        print (variables_here)
        this_function.update (variables_here=variables_here, variables_before=variables_here+variables_before, variables_after=[])
        
        if make_function:
            this_function.update_code ( 
                arguments=variables_before, 
                return_values=variables_here+variables_before,
                tab_size=tab_size
            )
            
        for function in self.function_list[:idx]:
            function.variables_after += [v for v in this_function.variables_here if v not in function.variables_after]
            if update_previous_functions and function.discover_outputs:
                this_function.update_code (function, return_values=function.variables_after, tab_size=tab_size)
    
    def write (self):
        with open (str(self.file_path), 'w') as file:
            for function in self.function_list:
                function.write (file)
                
    def print (self):
        for function in self.function_list:
            function.print ()
        
    def get_lib_path (self):
        return nbdev.config.get_config()['lib_path']
                   
    def get_nbs_path (self):
        return nbdev.config.get_config()['nbs_path']

In [ ]:
#| export
@magics_class
class CellProcessorMagic (Magics):
    """
    Base magic class for converting cells to modular functions.
    """
    def __init__(self, shell, **kwargs):
        super().__init__(shell)
        self.processor = CellProcessor (magic=self, **kwargs)
        
    @cell_magic
    def cell2file (self, folder, cell):
        self.processor.cell2file (folder, cell)
    
    @cell_magic
    def function (self, func, cell):
        "Converts cell to function"
        self.processor.function (func, cell)
    
    @line_magic
    def write (self, line):
        return self.processor.write ()
    
    @line_magic
    def print (self, line):
        return self.processor.print ()
        
    @line_magic
    def cell_processor (self, line):
        return self.processor

    #@magic_arguments()
    #@argument ('-o','--output', help='function output')
    #@argument ('arg', type=str, help='argument')
    @cell_magic
    def other (self, line, cell):
        if False:
            argv = shlex.split(line, posix=(os.name == 'posix'))
            for idx, arg in enumerate(argv):
                if arg and arg.startswith('-') and arg != '-' and arg != '->':
                    break
            print ('argv from idx', argv[idx:])
            #signature = ' '.join(shlex.quote(arg) for arg in argv[1:idx])
            signature = ' '.join(argv[1:idx])
            print ('signature', signature)
            if '(' in signature and ')' in signature:
                func, parameters = 'fun (a, b, c)'.split('(')
                parameters,_ = parameters.split(')')
                parameters = parameters.split(',')
                func = func.split()[0]
                print ('func', func, 'parameters', parameters)
            if '->' in signature:
                _, output = signature.split('->')
                output = output.split (',')
                print ('output', output)

            parameter_s = ' '.join(shlex.quote(arg) for arg in argv[idx:])
            print ('parameter_s', parameter_s)

            # get arguments and set sys.argv for program to be run.
            opts, arg_lst = self.parse_options(parameter_s,
                                               "n:r:t:q",
                                               mode='list', list_all=1,
                                               strict=False, preserve_non_opts=True)
            print ('opts from paramater_s', opts)
            print ('arg_lst from paramater_s', arg_lst)

            opts, stmt = self.parse_options(
                line, "n:r:t:q", posix=False, strict=False, preserve_non_opts=True
            )
            print ('opts from line', opts)
            print ('stmt from line', stmt)

In [ ]:
#| export
#| hide
def load_ipython_extension(ipython):
    """
    This module can be loaded via `%load_ext core.cell2func` or be configured to be autoloaded by IPython at startup time.
    """
    magics = CellProcessorMagic(ipython)
    ipython.register_magics(magics)

In [ ]:
#| export
def keep_variables (function, field, variable_values, self=None):
    """
    Store `variables` in dictionary entry `self.variables_field[function]`
    """
    frame_number = 1
    while not isinstance (self, CellProcessor):
        fr = sys._getframe(frame_number)
        args = argnames(fr, True)
        if len(args)>0:
            self = fr.f_locals[args[0]]
        frame_number += 1
    variable_values = {k: variable_values[k] for k in variable_values if not k.startswith ('_') and not callable(variable_values[k])}
    function_info = getattr(self, 'function_info')
    function_info[function][field]=variable_values

In [ ]:
load_ipython_extension (get_ipython())

In [ ]:
%%other fun (a, b, c) -> d, e, f -n "(a, b, c) -> o d, e, f" -t jajaj yes
print ('yes')

argv from idx ['-n', '(a, b, c) -> o d, e, f', '-t', 'jajaj', 'yes']
signature (a, b, c) -> d, e, f
func fun parameters ['a', ' b', ' c']


AttributeError: 'list' object has no attribute 'split'

In [ ]:
%%other fun -n a, b, c 
print ('yes')

argv from idx ['-n', 'a,', 'b,', 'c']
parameter_s -n a, b, c
opts from paramater_s {'n': ['a,']}
arg_lst from paramater_s ['b,', 'c']
opts from line {}
stmt from line fun -n a, b, c


In [ ]:
import ast

In [ ]:
%%function final
a = 1
b = 2
print (f'a + b is {a+b}')
c = a*b
print (f'a * b is {c}')

In [ ]:
%print all

In [ ]:
a = %cell_processor 1

In [ ]:
a.file_name

In [ ]:
final ()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()